# Step 0. Parameter Setting

In [1]:
#import dtnmonitor_v2 as dtn
import dtnmonitor_v3
dtn = dtnmonitor_v3.DTNMonitor()

import os,sys


### P1. SELECT THE MONITOR NETWORK INTERFACE
## used in Step 1,2,3 , if you don't know the interface name, you can use 'all'
## make sure the interface is correct, error_naming or without network flow will get empty grath
#dtn.interface = 'eth0.1301'
#dtn.interface = 'eth0.2038' 

dtn.interface = 'ens1f0.2038'
#dtn.interface = 'eth1'
#dtn.interface = 'all'


### P3. Iperf3 SERVER IP 
## used in Step 2,3 : input iperf3 server's ip
iperf3_server="10.250.38.52"

### P4. Iperf3 THREADS
## used in Step 2,3 :input threads number
count = 4

test_time="20" # second

window_size="1M" # M,K

## run fio and check status
### select graph_mode [0|1] 
#graph_mode=0  # all, 
#graph_mode=1  # 100pt
graph_mode=0  

### Show Disk and Interface
import netifaces
import os
import subprocess
import re

list=netifaces.interfaces()
print("==========================================================")
print('{:40s} {:20s} '.format("Network Interface","IP"))
print("==========================================================")
for inf in list:
    #print(inf)
    ff=netifaces.ifaddresses(inf)
    #print(ff)
    try:
        addr=ff[netifaces.AF_INET][0]['addr']
    except:
        continue
    print('{:40s} {:20s} '.format(inf,addr)) 

    
import getpass
user=getpass.getuser()

def exec_print(command):
    process = subprocess.Popen([command], stdout=subprocess.PIPE,stderr=subprocess.STDOUT, shell=True)
    for line in iter(process.stdout.readline, b''):
        # system print
        sys.stdout.write(line)

Network Interface                        IP                   
lo                                       127.0.0.1            
enp20s0                                  165.124.33.143       
ens1f0.2038                              10.250.38.50         
ens1f0.3061                              192.168.61.50        
ens1f1                                   192.168.1.1          
ens4.1301                                10.10.14.251         
ens4.3060                                192.168.60.50        


In [2]:

print("==========================================================")
print('{:60s} '.format("Initial checking"))
print("==========================================================")

command="ps aux |grep 'python3 '|wc -l"
process = subprocess.Popen([command], stdout=subprocess.PIPE,stderr=subprocess.STDOUT, shell=True)
ret=process.stdout.read().decode('utf8')



if int(ret) > 2: 
    print("System check ..... [ok]")
    print("Error!! Some process are running, please check first!!!")
    exec_print("ps -u "+ user +" ux |grep 'python3 '")
else : 
    print('Your Monitoring Network Interface ..... {:40s} '.format(dtn.interface))
    print("System check ..... [ok]")

Initial checking                                             
Your Monitoring Network Interface ..... ens1f0.2038                              
System check ..... [ok]


# Step 2. Test network transfer by memory

In [ ]:
command = ""
for i in range(count):
     command += "iperf3 -w "+window_size+" -Z -t "+test_time+"s -p 5500"+str(i + 1)+ " -c "+ iperf3_server +" &"
#print(command)
os.system(command)

dtn.run_monitor(timeout=int(test_time) + 5 )
#test_time=str(int(test_time)+20)
#dtn.exec_command("sleep "+test_time,1)



## Step A . stop the Iperf3 server

In [3]:
#dtn.stop()

#exec_print("killall -u "+user+" iperf3")

256

## Step B . check Iperf3 server status

In [ ]:
exec_print("ps -u "+ user +" ux |grep iperf3")